# SIFT FEATURE EXTRACTION

# ![https://media.licdn.com/dms/image/D5622AQF9GK6WowHAWA/feedshare-shrink_800/0/1690963072386?e=1694044800&v=beta&t=oDdP9MnUE4JYQXSuInpnoTGcCaqdZVHQ_4Kmv9Dj9Xk](https://media.licdn.com/dms/image/D5622AQF9GK6WowHAWA/feedshare-shrink_800/0/1690963072386?e=1694044800&v=beta&t=oDdP9MnUE4JYQXSuInpnoTGcCaqdZVHQ_4Kmv9Dj9Xk)

# ALL WE NEED

In [1]:
import cv2
import numpy as np
import os

# GET THE COUNT OF MATCHED KEYPOINTS

In [2]:
def get_matched_keypoint_count(query_image_path, candidate_image_path):
    query_image = cv2.imread(query_image_path, cv2.IMREAD_GRAYSCALE)
    candidate_image = cv2.imread(candidate_image_path, cv2.IMREAD_GRAYSCALE)
    sift = cv2.SIFT_create()
    query_keypoints, query_descriptors = sift.detectAndCompute(query_image, None)
    candidate_keypoints, candidate_descriptors = sift.detectAndCompute(candidate_image, None)
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(query_descriptors, candidate_descriptors, k=2)
    good_matches = []
    for m, n in matches:
        if m.distance < 0.7 * n.distance:
            good_matches.append(m)
    return len(good_matches)

# DRAW MATCHED KEYPOINTS

In [32]:
def draw_matched_keypoints(query_image_path, candidate_image_path, predicted_class):
    query_image_rgb = cv2.imread(query_image_path)
    candidate_image_rgb = cv2.imread(candidate_image_path)
    query_image_gray = cv2.cvtColor(query_image_rgb, cv2.COLOR_BGR2GRAY)
    candidate_image_gray = cv2.cvtColor(candidate_image_rgb, cv2.COLOR_BGR2GRAY)
    
    sift = cv2.SIFT_create()
    query_keypoints, query_descriptors = sift.detectAndCompute(query_image_gray, None)
    candidate_keypoints, candidate_descriptors = sift.detectAndCompute(candidate_image_gray, None)
    
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(query_descriptors, candidate_descriptors, k=2)
    
    good_matches = []
    for m, n in matches:
        if m.distance < 0.795 * n.distance:
            good_matches.append(m)
    
    matched_image = cv2.drawMatches(query_image_rgb, query_keypoints, candidate_image_rgb, candidate_keypoints, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    
    for match in good_matches:
        query_keypoint = query_keypoints[match.queryIdx].pt
        candidate_keypoint = candidate_keypoints[match.trainIdx].pt
        query_keypoint = tuple(map(int, query_keypoint))
        candidate_keypoint = tuple(map(int, candidate_keypoint))
        cv2.line(matched_image, query_keypoint, (candidate_keypoint[0] + query_image_rgb.shape[1], candidate_keypoint[1]), (0, 255, 0), 1)
    
    # Draw predicted class in a black rectangle at the center bottom
    font = cv2.FONT_HERSHEY_SIMPLEX
    text = f"PREDICTED : {predicted_class}"
    text_size, _ = cv2.getTextSize(text, font, 1, 2)
    text_width, text_height = text_size
    rect_height = text_height + 10
    rect_y = matched_image.shape[0] - rect_height
    cv2.rectangle(matched_image, (0, rect_y), (matched_image.shape[1], matched_image.shape[0]), (0, 0, 0), -1)
    text_position = ((matched_image.shape[1] - text_width) // 2, matched_image.shape[0] - 5)
    cv2.putText(matched_image, text, text_position, font, 1, (0, 255, 0), 2)

    cv2.imshow("Matched Image", matched_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return len(good_matches)


# INVOKE

In [35]:
def main():
    test_folder = './Images/SIFT/TESTS/'
    candidate_folder = './Images/SIFT/CANDIDATES/'
    test_image_paths = []
    test_image_labels = []
    for class_name in os.listdir(test_folder):
        class_folder = os.path.join(test_folder, class_name)
        if not os.path.isdir(class_folder):
            continue
        test_image_paths.extend([os.path.join(class_folder, img_name) for img_name in os.listdir(class_folder)])
        test_image_labels.extend([class_name] * len(os.listdir(class_folder)))

    correct_predictions = 0
    total_test_images = len(test_image_paths)

    for i, test_image_path in enumerate(test_image_paths):
        print(f"Test image {i+1}/{total_test_images}: {test_image_path}")

        max_matches = 0
        best_candidate = ""
        best_class_name = ""

        for class_name in os.listdir(candidate_folder):
            class_folder = os.path.join(candidate_folder, class_name)
            if not os.path.isdir(class_folder):
                continue

            candidate_image_paths = [os.path.join(class_folder, img_name) for img_name in os.listdir(class_folder)]

            for candidate_image_path in candidate_image_paths:
                matched_keypoint_count = get_matched_keypoint_count(test_image_path, candidate_image_path)
                if matched_keypoint_count > max_matches:
                    max_matches = matched_keypoint_count
                    best_candidate = candidate_image_path
                    best_class_name = class_name

        #print(f"Best match from class '{best_class_name}' with {max_matches} keypoints: {best_candidate}")
        print("---")

        # Display the image with the maximum matches (optional)

        # Check correctness and calculate accuracy
        predicted_class = best_class_name
        draw_matched_keypoints(test_image_path, best_candidate,predicted_class)
        ground_truth_class = test_image_labels[i]
        if predicted_class == ground_truth_class:
            print(predicted_class)
            correct_predictions += 1

        accuracy = (correct_predictions / total_test_images) * 100
        print(f"Accuracy: {accuracy:.2f}%")

if __name__ == "__main__":
    main()


Test image 1/4: ./Images/SIFT/TESTS/GOLDFISH/Screenshot from 2023-08-04 19-07-38.png
---
GOLDFISH
Accuracy: 25.00%
Test image 2/4: ./Images/SIFT/TESTS/PYTHON/Screenshot from 2023-08-04 19-29-47.png
---
PYTHON
Accuracy: 50.00%
Test image 3/4: ./Images/SIFT/TESTS/PARAKEET/Screenshot from 2023-08-04 19-14-00.png
---
PARAKEET
Accuracy: 75.00%
Test image 4/4: ./Images/SIFT/TESTS/PENGUIN/Screenshot from 2023-08-04 19-13-00.png
---
PENGUIN
Accuracy: 100.00%
